In [1]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

import cufflinks as cf

import pandas as pd
import numpy as np

import geojson


import sqlite3
from sqlalchemy import create_engine

In [251]:
pd.set_option('max_colwidth', 3000)
pd.set_option('max_columns', 300)

In [41]:
(
    prices['count']
    .apply(lambda x: x / x.sum(), axis = 1)
    .sort_values(by='Entire home/apt')
    .iplot(kind='barh', filename = 'paris_airbnb_props', 
           layout = graph_objs.Layout(title = 'Entire apartment is the most common type of accommodation', 
                                      barmode = 'stack', hovermode = 'closest', 
                                      xaxis = dict(tickformat = '%a')
                                     )
          )
)

In [53]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [57]:
(
    listings
    .groupby(['neighbourhood'])
    .price
    .agg(['count', 'min', 'max', 'mean', percentile(25), 'median', percentile(75)])
)

,count,min,max,mean,percentile_25,median,percentile_75
neighbourhood,,,,,,,
Batignolles-Monceau,3897,10,5000,106.126764,55,75,110
Bourse,1937,10,1850,136.515230,74,100,150
Buttes-Chaumont,3271,0,5112,77.889025,45,60,84
Buttes-Montmartre,6626,0,9379,82.377452,50,69,90
Entrepôt,4056,9,1974,95.614152,55,77,104
Gobelins,2057,0,5000,84.033544,50,68,95
Hôtel-de-Ville,1856,10,1550,151.918642,80,120,170
Louvre,1116,9,5000,178.577061,86,120,200
Luxembourg,1720,9,2500,172.656977,80,119,195


In [7]:
prices = (
    listings
    .groupby(['neighbourhood', 'room_type'])
    .agg(['count', 'min', 'max', 'mean', 'median'])
    .price
    .unstack()
)
prices

count                                      min  \
room_type           Entire home/apt Private room Shared room Entire home/apt   
neighbourhood                                                                  
Batignolles-Monceau            3402          473          22              10   
Bourse                         1725          192          20              10   
Buttes-Chaumont                2685          544          42               0   
Buttes-Montmartre              5874          710          42               0   
Entrepôt                       3471          541          44               9   
Gobelins                       1592          444          21               0   
Hôtel-de-Ville                 1690          158           8              10   
Louvre                          962          148           6               9   
Luxembourg                     1490          227           3              28   
Ménilmontant                   2858          557          36              10   
Observatoire                   1821          347          47              12   
Opéra                          2247          397          13               8   
Palais-Bourbon                 1480          156           6              10   
Panthéon                       1715          246          15              20   
Passy                          2558          324          21               0   
Popincourt                     5053          619          59               0   
Reuilly                        1973          340          30               0   
Temple                         2597          198           6               0   
Vaugirard                      3692          587          42               0   
Élysée                         1287          197           7              10   

                                                         max               \
room_type           Private room Shared room Entire home/apt Private room   
neighbourhood                                                               
Batignolles-Monceau           15          20            2000         1000   
Bourse                        19          21            1850         1000   
Buttes-Chaumont               10          10            5112         1000   
Buttes-Montmartre              8          14            9379         1000   
Entrepôt                       9          14            1974         1200   
Gobelins                      15          15            5000         1000   
Hôtel-de-Ville                20          41            1550         1000   
Louvre                        29          30            5000         1000   
Luxembourg                     9          45            2097         2500   
Ménilmontant                  17          10            1300         1000   
Observatoire                   9          20            3500         8738   
Opéra                         13          25            1350        25000   
Palais-Bourbon                25          26            1972         1000   
Panthéon                      10          15            1900         1000   
Passy                          8          22            8721         1250   
Popincourt                     9          15            2000          360   
Reuilly                       16          12            1000         1000   
Temple                        22          25            8500         1975   
Vaugirard                     13          15            8576         1090   
Élysée                        12          50            3000        25000   

                                           mean                           \
room_type           Shared room Entire home/apt Private room Shared room   
neighbourhood                                                              
Batignolles-Monceau        5000      101.355379   131.126850  306.454545   
Bourse                       92      136.450435   147.682292   34.900000   
Buttes-Chaumont              65      

In [40]:
(
    prices['count']
    .apply(lambda x: x / x.sum(), axis = 1)
    .sort_values(by='Entire home/apt', ascending = False)
    .iplot(kind='barh', filename = 'paris_airbnb_props', 
           layout = graph_objs.Layout(title = 'Entire apartment is the most common type of accommodation', 
                                      barmode = 'stack', hovermode = 'closest', 
                                      xaxis = dict(tickformat = '%a')
                                     )
          )
)

In [59]:
listings.head(1)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2577,Loft for 4 by Canal Saint Martin,2827,Karine,NaN,Entrepôt,48.869933,2.362511,Entire home/apt,125,3,1,2019-01-02,0.86,1,85


In [63]:
(
    listings
    .query("last_review > '2019-01-01'")
    .count()
)

id                                15248
name                              15244
host_id                           15248
host_name                         15248
neighbourhood_group                   0
neighbourhood                     15248
latitude                          15248
longitude                         15248
room_type                         15248
price                             15248
minimum_nights                    15248
number_of_reviews                 15248
last_review                       15248
reviews_per_month                 15248
calculated_host_listings_count    15248
availability_365                  15248
dtype: int64

In [87]:
calendar_feb = pd.read_csv('calendar.csv')

In [88]:
calendar_feb.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2577,2019-02-05,f,$125.00,$125.00,3.0,1125.0
1,2577,2019-02-06,f,$125.00,$125.00,3.0,1125.0
2,2577,2019-02-07,f,$125.00,$125.00,3.0,1125.0
3,2577,2019-02-08,f,$125.00,$125.00,3.0,1125.0
4,2577,2019-02-09,f,$125.00,$125.00,3.0,1125.0


In [13]:
calendar_feb.count()

listing_id        21195164
date              21195164
available         21195164
price             21194984
adjusted_price    21194984
minimum_nights    21195147
maximum_nights    21195147
dtype: int64

In [14]:
calendar_feb.date.max()

'2020-02-05'

In [15]:
(
    calendar_feb
    .query('listing_id == 2577')
    .count()
    .head(1)
)

listing_id    365
dtype: int64

In [16]:
len(
    calendar_feb
    .listing_id
    .unique()
)

58067

In [17]:
(
    calendar_feb
    [['listing_id', 'available']]
    .groupby('listing_id')
    .max()
    .query("available == 'f'")
    .count()
)

available    26714
dtype: int64

In [2]:
engine = create_engine('sqlite:///liligo.db', echo=False)

In [8]:
pd.read_sql_query('''
    select is_available, count(*) as cnt
    from (
        select listing_id, max(available) as is_available
        from calendar_feb
        group by listing_id
        ) t
    group by is_available
''', engine)

,is_available,cnt
0,f,26714
1,t,31353


In [9]:
pd.read_sql_query('''
    select count(*) as cnt
    from (
        select listing_id, available, count(*) as num_available
        from calendar_feb
        group by listing_id, available
        ) t
    where 1=1
        and available = 't'
        and num_available > 10
''', engine)

,cnt
0,27502


In [10]:
pd.read_sql_query('''
    select count(*) as cnt
    from (
        select listing_id, available, count(*) as num_available
        from calendar_feb
        group by listing_id, available
        ) t
    where 1=1
        and available = 't'
        and num_available > 30
''', engine)

,cnt
0,24372


In [232]:
(
    pd
    .DataFrame([('feb', 9324), ('jan', 12382), ('dec', 15587), ('nov', 14600), ('oct', 16456)], columns = ['month', '3 weeks ahead']).set_index('month')
    .join( pd.DataFrame([('feb', 7531), ('jan', 11213), ('dec', 14732), ('nov', 14094), ('oct', 16030)], columns = ['month', '2 weeks ahead']).set_index('month'))
    .join( pd.DataFrame([('feb', 6633), ('jan', 11143), ('dec', 14600), ('nov', 14115), ('oct', 15961)], columns = ['month', '<1 week ahead']).set_index('month'))
    [::-1]
    .iplot(filename = 'paris_counts', kind = 'bar', 
           layout = graph_objs.Layout(title = '<b>Number of available places drop quickly in the last 4 weeks before the trip</b><br>Number of listings for a February weekend in Paris at different times', 
                                      hovermode = 'closest')
          )
)